In [1]:
import pandas as pd

books = pd.read_csv("./datasets/books_with_categories.csv")

In [15]:
from transformers import pipeline

classifier = pipeline(
    "text-classification",
    model = "j-hartmann/emotion-english-distilroberta-base",
    top_k = None,
    device=0
)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at j-hartmann/emotion-english-distilroberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
Device set to use 0


In [16]:
classifier("I love this!")

[[{'label': 'joy', 'score': 0.9771831035614014},
  {'label': 'surprise', 'score': 0.008522207848727703},
  {'label': 'neutral', 'score': 0.005764706060290337},
  {'label': 'anger', 'score': 0.004416599869728088},
  {'label': 'sadness', 'score': 0.0020901544485241175},
  {'label': 'disgust', 'score': 0.0016098330961540341},
  {'label': 'fear', 'score': 0.00041336481808684766}]]

In [17]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [18]:
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.6546940803527832},
  {'label': 'neutral', 'score': 0.16979697346687317},
  {'label': 'sadness', 'score': 0.11659283936023712},
  {'label': 'surprise', 'score': 0.020702548325061798},
  {'label': 'disgust', 'score': 0.019115153700113297},
  {'label': 'joy', 'score': 0.015163161791861057},
  {'label': 'anger', 'score': 0.0039353142492473125}]]

In [22]:
predictions = classifier(books["description"][0].split("."))

In [23]:
sentence = books["description"][0].split(".")
sentence[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [24]:
predictions[0]

[{'label': 'surprise', 'score': 0.7296899557113647},
 {'label': 'neutral', 'score': 0.14041072130203247},
 {'label': 'fear', 'score': 0.06813927739858627},
 {'label': 'joy', 'score': 0.047856222838163376},
 {'label': 'anger', 'score': 0.009153462015092373},
 {'label': 'disgust', 'score': 0.0026283222250640392},
 {'label': 'sadness', 'score': 0.002121988218277693}]

In [27]:
sentence[3]

' Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist'

In [28]:
predictions[3]

[{'label': 'fear', 'score': 0.9279921054840088},
 {'label': 'anger', 'score': 0.03229950740933418},
 {'label': 'neutral', 'score': 0.0128178671002388},
 {'label': 'sadness', 'score': 0.008772886358201504},
 {'label': 'surprise', 'score': 0.008618028834462166},
 {'label': 'disgust', 'score': 0.008454025723040104},
 {'label': 'joy', 'score': 0.001045519020408392}]

In [30]:
sorted(predictions[0], key=lambda x: x["label"])

[{'label': 'anger', 'score': 0.009153462015092373},
 {'label': 'disgust', 'score': 0.0026283222250640392},
 {'label': 'fear', 'score': 0.06813927739858627},
 {'label': 'joy', 'score': 0.047856222838163376},
 {'label': 'neutral', 'score': 0.14041072130203247},
 {'label': 'sadness', 'score': 0.002121988218277693},
 {'label': 'surprise', 'score': 0.7296899557113647}]

In [35]:
import numpy as np

emotion_labels = ["anger", "disgust", "fear", "joy", "neutral", "sadness", "surprise"]
isbn = [] 
emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictiosn):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])

    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [38]:
from tqdm import tqdm

for i in tqdm(range(10)):
    isbn.append(books["isbn13"][i])
    sentences= books["description"][i].split()
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|████████████████████████████████████████████████████████████████████████████████████████| 10/10 [02:34<00:00, 15.42s/it]


In [39]:
emotion_scores

{'anger': [0.9597889184951782,
  0.8685598969459534,
  0.9098340272903442,
  0.2986922562122345,
  0.4047142565250397,
  0.1822400540113449,
  0.7623999118804932,
  0.33344241976737976,
  0.636991024017334,
  0.1930156648159027],
 'disgust': [0.9547021389007568,
  0.6384161710739136,
  0.43453630805015564,
  0.4200974106788635,
  0.23651501536369324,
  0.569498598575592,
  0.4315760135650635,
  0.11937256902456284,
  0.6588069796562195,
  0.1866736263036728],
 'fear': [0.8833518624305725,
  0.5188772082328796,
  0.9310702681541443,
  0.9438542127609253,
  0.2750580310821533,
  0.3342118263244629,
  0.9232286810874939,
  0.452020525932312,
  0.36169230937957764,
  0.24676461517810822],
 'joy': [0.9695032238960266,
  0.47738921642303467,
  0.9367653131484985,
  0.7361072897911072,
  0.1864885538816452,
  0.12233678996562958,
  0.974264919757843,
  0.8970750570297241,
  0.9367653131484985,
  0.14794574677944183],
 'neutral': [0.9341217875480652,
  0.94412761926651,
  0.8921584486961365,
 

In [ ]:
from tqdm import tqdm

emotion_labels = ["anger", "disgust", "fear", "joy", "neutral", "sadness", "surprise"]
isbn = [] 
emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences= books["description"][i].split()
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

  0%|                                                                                    | 1/5197 [00:32<46:14:28, 32.04s/it]

In [ ]:
emotion_df = pd.DataFrame(emotion_scores)
emotion_df["isbn13"] = isbn

In [ ]:
books = pd.merge(books,emotion_df, on="isbn13")

In [ ]:
books.to_csv("books_with_emotions.csv")